In [4]:
import sys
import os
import json

# Cari path ke project_root dari lokasi notebook
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Tambahkan project root ke sys.path jika belum ada
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
DB_USERNAME = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "dvdrental"


from utils.database import Database

db = Database("postgresql", DB_HOST, DB_PORT, DB_USERNAME, DB_PASSWORD, DB_NAME, "public")

db_metadata = db.impl.get_load_status()
db_metadata

{'public': [{'table': 'actor', 'incremental': False},
  {'table': 'store', 'incremental': False},
  {'table': 'address', 'incremental': False},
  {'table': 'category', 'incremental': False},
  {'table': 'city', 'incremental': False},
  {'table': 'country', 'incremental': False},
  {'table': 'film_actor', 'incremental': False},
  {'table': 'film_category', 'incremental': False},
  {'table': 'inventory', 'incremental': False},
  {'table': 'language', 'incremental': False},
  {'table': 'payment', 'incremental': True},
  {'table': 'staff', 'incremental': False},
  {'table': 'customer', 'incremental': True},
  {'table': 'rental', 'incremental': True},
  {'table': 'film', 'incremental': True}]}

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa 
import io
from minio import Minio
from datetime import datetime


client = Minio(
        "localhost:9000",
        access_key="admin_minio",
        secret_key="admin_minio",
        secure=False
    )



for schema, table_list in db_metadata.items():
    for table in table_list:
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        
        df = pd.read_sql(f"SELECT * FROM {schema}.{table['table']}", db.impl.engine)
        
        # pq.write_table(pa.Table.from_pandas(df), f"{table['table']}.parquet", compression="snappy")
        
        buffer = io.BytesIO()
        pq.write_table(pa.Table.from_pandas(df), buffer, compression="snappy")
        buffer.seek(0)
        
        # client.put_object(
        #     bucket_name="landing-zones",
        #     object_name=f"{DB_NAME}/{schema}/{table['table']}/{current_date}/{table['table']}.parquet",
        #     data=buffer,
        #     length=len(buffer.getvalue()),
        #     content_type="application/vnd.apache.parquet"
        # )

application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream
application/octet-stream


In [3]:
print(buffer.getvalue()[:4]) 

b'PAR1'


In [ ]:
# GET DATA LOAD ALL

import pandas as pd

df_chunks = pd.read_sql("SELECT * FROM category", db.impl.engine, chunksize=10000)

for df in df_chunks:
    print(df.dtypes)
    break

In [ ]:
# GET DATA LOAD INCREMENTAL

import pandas as pd 
from datetime import datetime

current_date = datetime.now().strftime("%Y-%m-%d")

df = pd.read_sql(f"SELECT * FROM inventory where updated_at >= '{current_date}'", db.impl.engine)
print(df.dtypes)

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd 
import io


df = pd.read_sql(f"SELECT * FROM film", db.impl.engine)
# print(df.dtypes)

pq.write_table(pa.Table.from_pandas(df), "film.parquet", compression="snappy")

# buffer = io.BytesIO()
# pq.write_table(pa.Table.from_pandas(df), buffer, compression="snappy")
# buffer.seek(0)

In [ ]:
from minio import Minio
from minio.error import S3Error

client = Minio(
        "localhost:9000",
        access_key="admin_minio",
        secret_key="admin_minio",
        secure=False
    )

client.put_object(
        bucket_name="data",
        object_name="public/staff.parquet",
        data=buffer,
        length=len(buffer.getvalue()),
        content_type="application/vnd.apache.parquet"
    )